The tough problem when we are querying classes for a type variable is capturing *user intent* - should that type variable be assigned to a specific class, or a general protocol?

Our assumptions:

- If one non-None instance is associated with the type variable at runtime, we assume that the type variable should be assigned to the class of that instance,
- If zero or many non-None instances are associated with the type variable at runtime, we assume that the type variable should be assigned to a general protocol.

Thus:

- If one non-None instance is associated with the type variable at runtime, we return only the class of that type variable as the query result.
- If zero or many non-None instances are associated with the type variable at runtime, we query classes using TF-IDF.

When querying classes using TF-IDF, there are certain aspects we need to pay attention to:

- Only classes that the user can access should be within the query database (this excludes classes such as `builtins.dict_keys`).
- No two classes should have overlapping attribute sets.
- Two classes with similar TF-IDF embeddings should be similar in semantic meaning, i.e., generally replacable. This is to enhance the robustness of predictions, especially given the fact that query vectors tend to be sparse.

We initialize the query database with classes and general protocols that are considered unconditionally available. These include the types in the following modules:

- `builtins`
- `typing`
- `collections.abc`
- `numbers`
- `_typeshed`

They cover everything in Python's [Standard Type Hierarchy](https://docs.python.org/3/reference/datamodel.html):

```
3.2.1. None
3.2.2. NotImplemented
3.2.3. Ellipsis
3.2.4. numbers.Number
    3.2.4.1. numbers.Integral
    3.2.4.2. numbers.Real
    3.2.4.3. numbers.Complex
3.2.5. Sequences
    3.2.5.1. Immutable sequences
        Strings
        Tuples
        Bytes
    3.2.5.2. Mutable sequences
        Lists
        Byte Arrays
3.2.6. Set types
    Sets
    Frozen sets
3.2.7 Mappings
    3.2.7.1 Dictionaries
3.2.8 Callable types
    3.2.8.1. User-defined functions
    3.2.8.2. Instance method
    3.2.8.3. Generator functions
    3.2.8.4. Coroutine functions
    3.2.8.5. Asynchronous generator functions
    3.2.8.6. Built-in functions
    3.2.8.7. Built-in methods
    3.2.8.8. Classes
    3.2.8.9. Class Instances
3.2.9. Modules
3.2.10. Custom classes
3.2.11. Class instances
3.2.12. I/O objects (also known as file objects)
3.2.13. Internal types
    3.2.13.1. Code objects
    3.2.13.2. Frame objects
    3.2.13.3. Traceback objects
    3.2.13.4. Slice objects
    3.2.13.5. Static method objects
    3.2.13.6. Class method objects
```

For the classes and general protocols in these modules, due to their complexity, we manually select those to add to the query database based on the criterion above.

For classes in other modules, build inheritance graphs, and only add classes which add new attributes to at least one of its base classes.

In [1]:
import builtins
import collections.abc
import numbers
import typing

modules = [
    builtins,
    collections.abc,
    numbers,
    typing
]

In [2]:
from typeshed_client_ex.client import Client

In [3]:
from typeshed_client_ex.type_definitions import *

In [4]:
client = Client()

In [5]:
from get_types_in_module import get_types_in_module

In [6]:
from get_attributes_in_runtime_class import get_attributes_in_runtime_class

In [7]:
import logging

In [8]:
from collections import defaultdict

typeshed_class_to_attribute_frozenset: dict[TypeshedClass, frozenset[str]] = dict()
attribute_frozenset_to_typeshed_class_set: defaultdict[frozenset[str], set[TypeshedClass]] = defaultdict(set)

for module in modules:
    runtime_classes_in_module = get_types_in_module(module)
    for runtime_class in runtime_classes_in_module:
        runtime_class_attribute_set = get_attributes_in_runtime_class(runtime_class)
        typeshed_class = from_runtime_class(runtime_class)

        try:
            typeshed_class_definition = client.get_class_definition(typeshed_class)
            typeshed_class_attribute_set = get_attributes_in_class_definition(typeshed_class_definition)
            # attribute_frozenset: frozenset[str] = frozenset(runtime_class_attribute_set & typeshed_class_attribute_set)
        except:
            # attribute_frozenset: frozenset[str] = frozenset(runtime_class_attribute_set)
            logging.exception('Failed to retrieve Typeshed class definition for %s.', typeshed_class)
        
        attribute_frozenset: frozenset[str] = frozenset(runtime_class_attribute_set)

        typeshed_class_to_attribute_frozenset[typeshed_class] = attribute_frozenset
        if attribute_frozenset in attribute_frozenset_to_typeshed_class_set:
            attribute_frozenset_to_typeshed_class_set[attribute_frozenset].add(typeshed_class)
        else:
            attribute_frozenset_to_typeshed_class_set[attribute_frozenset] = {typeshed_class}
        
for typeshed_class, typeshed_class_definition in client.get_all_class_definitions_in_module('_typeshed').items():
    attribute_frozenset: frozenset[str] = frozenset(get_attributes_in_class_definition(typeshed_class_definition) | get_attributes_in_runtime_class(object))
    typeshed_class_to_attribute_frozenset[typeshed_class] = attribute_frozenset
    if attribute_frozenset in attribute_frozenset_to_typeshed_class_set:
        attribute_frozenset_to_typeshed_class_set[attribute_frozenset].add(typeshed_class)
    else:
        attribute_frozenset_to_typeshed_class_set[attribute_frozenset] = {typeshed_class}


ERROR:root:Failed to retrieve Typeshed class definition for collections.abc._CallableGenericAlias.
Traceback (most recent call last):
  File "/tmp/ipykernel_7267/3771127605.py", line 13, in <module>
    typeshed_class_definition = client.get_class_definition(typeshed_class)
  File "/home/jifengwu/name_scope_resolution/typeshed_client_ex/client.py", line 835, in get_class_definition
    raise AttributeError(
AttributeError: module 'collections.abc' has no attribute '_CallableGenericAlias'
ERROR:root:Failed to retrieve Typeshed class definition for typing._Final.
Traceback (most recent call last):
  File "/tmp/ipykernel_7267/3771127605.py", line 13, in <module>
    typeshed_class_definition = client.get_class_definition(typeshed_class)
  File "/home/jifengwu/name_scope_resolution/typeshed_client_ex/client.py", line 835, in get_class_definition
    raise AttributeError(
AttributeError: module 'typing' has no attribute '_Final'
ERROR:root:Failed to retrieve Typeshed class definition for ty

In [9]:
def attribute_frozenset_to_string(attribute_frozenset: frozenset[str]) -> str:
    return '\n'.join(attribute_frozenset)

In [10]:
def typeshed_class_set_to_string(typeshed_class_set: set[TypeshedClass]) -> str:
    return '\n'.join(
        (str(typeshed_class) for typeshed_class in typeshed_class_set)
    )

We calculate and visualize a *mutual nearest neighbor similarity graph* of the classes and general protocols in these modules.

In [11]:
attribute_frozenset_list: list[frozenset[str]] = list(attribute_frozenset_to_typeshed_class_set.keys())

In [12]:
# Add nodes to graph

In [13]:
import networkx as nx

G: nx.Graph = nx.Graph()

for index, _ in enumerate(attribute_frozenset_list):
    G.add_node(index)

In [14]:
# Get attribute frozenset IDF embeddings

In [15]:
from collections import Counter
from math import log

attribute_frequency_counter: Counter[str] = Counter()

for attribute_frozenset in attribute_frozenset_list:
    attribute_frequency_counter.update(attribute_frozenset)

attribute_list: list[str] = list(attribute_frequency_counter.keys())
attribute_to_index: dict[str, int] = {
    attribute: index
    for index, attribute in enumerate(attribute_list)
}
attribute_to_idf: dict[str, float] = {
    attribute: log(len(attribute_frozenset_list) / attribute_frequency)
    for attribute, attribute_frequency in attribute_frequency_counter.items()
}

In [16]:
from numpy import ndarray, zeros


def get_idf_ndarray(attribute_frozenset: frozenset[str]) -> ndarray:
    global attribute_to_index, attribute_to_idf
    
    idf_ndarray = zeros(len(attribute_to_index))
    for attribute in attribute_frozenset:
        try:
            index = attribute_to_index[attribute]
            idf = attribute_to_idf[attribute]
            idf_ndarray[index] = idf
        except KeyError:
            logging.warning('Skipped attribute: %s', attribute)
    return idf_ndarray

In [17]:
attribute_frozenset_idf_embeddings = zeros(
    (len(attribute_frozenset_list), len(attribute_to_index))
)

for index, attribute_frozenset in enumerate(attribute_frozenset_list):
    idf_ndarray = get_idf_ndarray(attribute_frozenset)
    attribute_frozenset_idf_embeddings[index] = idf_ndarray

In [18]:
attribute_frozenset_idf_embeddings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 4.63472899, 4.63472899,
        4.63472899]])

In [19]:
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

# Normalize data for cosine similarity
attribute_frozenset_idf_embeddings_normalized = normalize(attribute_frozenset_idf_embeddings, norm='l2')

# Compute cosine distances
cosine_distance_matrix = pairwise_distances(attribute_frozenset_idf_embeddings_normalized, attribute_frozenset_idf_embeddings_normalized, metric='cosine')

In [20]:
import numpy as np

# Get k-nearest neighbors
k = 5
k_nearest_indices_of_attribute_frozensets = np.argsort(cosine_distance_matrix)[:, 1:k+1]  # Excluding the diagonal

# Identify mutual k-nearest neighbors
mutual_neighbor_index_pairs = set()
for index, k_nearest_indices in enumerate(k_nearest_indices_of_attribute_frozensets):
    for k_nearest_index in k_nearest_indices:
        if index in k_nearest_indices_of_attribute_frozensets[k_nearest_index]:
            mutual_neighbor_index_pairs.add(tuple(sorted([index, k_nearest_index])))

In [21]:
mutual_neighbor_index_pairs

{(0, 11),
 (0, 12),
 (0, 24),
 (0, 26),
 (0, 30),
 (1, 29),
 (1, 31),
 (1, 43),
 (2, 23),
 (2, 38),
 (2, 48),
 (2, 84),
 (5, 59),
 (5, 60),
 (5, 77),
 (6, 22),
 (6, 55),
 (6, 56),
 (8, 14),
 (8, 20),
 (8, 34),
 (8, 36),
 (10, 15),
 (10, 16),
 (10, 17),
 (11, 12),
 (11, 24),
 (11, 26),
 (11, 30),
 (12, 24),
 (12, 26),
 (12, 30),
 (13, 14),
 (13, 18),
 (13, 40),
 (14, 34),
 (15, 16),
 (15, 17),
 (15, 52),
 (15, 59),
 (16, 17),
 (16, 77),
 (18, 21),
 (18, 27),
 (18, 40),
 (19, 22),
 (19, 53),
 (19, 54),
 (21, 27),
 (22, 53),
 (22, 55),
 (22, 56),
 (23, 48),
 (23, 84),
 (24, 26),
 (24, 30),
 (26, 30),
 (28, 41),
 (28, 51),
 (28, 97),
 (29, 31),
 (29, 35),
 (29, 43),
 (31, 35),
 (31, 43),
 (32, 33),
 (32, 34),
 (32, 38),
 (32, 46),
 (32, 47),
 (33, 42),
 (33, 47),
 (33, 92),
 (34, 36),
 (34, 40),
 (35, 42),
 (35, 43),
 (35, 47),
 (36, 38),
 (37, 45),
 (37, 49),
 (39, 50),
 (40, 41),
 (40, 97),
 (41, 51),
 (41, 95),
 (41, 97),
 (42, 47),
 (44, 65),
 (44, 67),
 (44, 83),
 (45, 49),
 (45, 85),

In [22]:
for first_index, second_index in mutual_neighbor_index_pairs:
    G.add_edge(first_index, second_index)

In [23]:
import pygraphviz as pgv

A: pgv.AGraph = nx.nx_agraph.to_agraph(G)

In [24]:
import matplotlib.pyplot as plt

# Compute the centrality of the nodes (using degree centrality in this case)
centrality = nx.degree_centrality(G)

# Normalize the centrality values to lie between 0 and 1
max_centrality = max(centrality.values())
normalized_centrality = {node: value / max_centrality for node, value in centrality.items()}

# Function to map centrality to color
def map_value_to_color(value, colormap=plt.cm.coolwarm):
    return colormap(value)

In [25]:
for node, centrality_value in normalized_centrality.items():
    color = map_value_to_color(centrality_value)
    
    # Convert matplotlib color to hex format for GraphViz
    hex_color = '#%02x%02x%02x' % (int(color[0]*255), int(color[1]*255), int(color[2]*255))

    A.get_node(node).attr['label'] = typeshed_class_set_to_string(
        attribute_frozenset_to_typeshed_class_set[attribute_frozenset_list[node]]
    )
    A.get_node(node).attr['style'] = 'filled'
    A.get_node(node).attr['fillcolor'] = hex_color

for first_index, second_index in mutual_neighbor_index_pairs:
    A.get_edge(first_index, second_index).attr['label'] = attribute_frozenset_to_string(
        attribute_frozenset_list[first_index].symmetric_difference(attribute_frozenset_list[second_index])
    )

In [26]:
A.draw("file.pdf", prog="dot")  # use circo to position, write PS file

In [27]:
from IPython.display import IFrame
IFrame("file.pdf", width=1024, height=768)

Based on the visualization, we include the following classes in `builtins`, `typing`, `collections.abc`, `numbers` into the class query dataset:

# Exception-related

The base class `builtins.BaseException` from the following classes with the same attribute set:

```
builtins.FutureWarning
builtins.BaseException
builtins.KeyboardInterrupt
builtins.ImportWarning
builtins.AssertionError
builtins.PendingDeprecationWarning
builtins.BufferError
builtins.RuntimeError
builtins.EOFError
builtins.ResourceWarning
builtins.RuntimeWarning
builtins.UnicodeError
builtins.ReferenceError
builtins.Exception
builtins.RecursionError
builtins.UnicodeWarning
builtins.ZeroDivisionError
builtins.TypeError
builtins.DeprecationWarning
builtins.Warning
builtins.FloatingPointError
builtins.StopAsyncIteration
builtins.EncodingWarning
builtins.SyntaxWarning
builtins.MemoryError
builtins.NotImplementedError
builtins.ArithmeticError
builtins.ValueError
builtins.OverflowError
builtins.BytesWarning
builtins.IndexError
builtins.SystemError
builtins.KeyError
builtins.LookupError
builtins.UserWarning
builtins.GeneratorExit
```

The base class`builtins.OSError` from the following classes with the same attribute set:

```
builtins.PermissionError
builtins.FileExistsError
builtins.BrokenPipeError
builtins.ConnectionResetError
builtins.ChildProcessError
builtins.ConnectionAbortedError
builtins.ProcessLookupError
builtins.InterruptedError
builtins.ConnectionError
builtins.OSError
builtins.BlockingIOError
builtins.NotADirectoryError
builtins.TimeoutError
builtins.FileNotFoundError
builtins.ConnectionRefusedError
builtins.IsADirectoryError
```

The base class `builtins.SyntaxError` from the following classes with the same attribute set:

```
builtins.IndentationError
builtins.SyntaxError
builtins.TabError
```

The base class `builtins.NameError` from the following classes with the same attribute set:

```
builtins.NameError
builtins.UnboundLocalError
```

The base class `builtins.ImportError` from the following classes with the same attribute set:

```
builtins.ImportError
builtins.ModuleNotFoundError
```

`builtins.AttributeError`.

`builtins.SystemExit`.

`builtins.StopIteration`.

Skipped:

```
builtins.UnicodeTranslateError
builtins.UnicodeDecodeError
builtins.UnicodeEncodeError
```

------

# Container-related

`builtins.list`.

`builtins.bytearray`.

`builtins.str`.

`builtins.bytes`.

`builtins.set`.

`builtins.frozenset`.

`builtins.dict`.

`builtins.tuple`.

`builtins.range`.

`builtins.slice`.

`builtins.type`, from the following classes with the same attribute set:

```
typing.NamedTupleMeta
builtins.type
typing._TypedDictMeta
```

`typing.Container`.

`typing.Sized`, which also assumes:

```
typing.MappingView
```

`typing.Iterable`.

`typing.Collection`, which also subsumes:

```
typing.ValuesView
```

`typing.Reversible`.

`typing.Iterator`, which also subsumes:

```
builtins.enumerate

builtins.filter
builtins.map

builtins.zip

builtins.reversed
```

`typing.Generator`.

`typing.AsyncIterable`.

`typing.AsyncIterator`.

`typing.AsyncGenerator`.

`typing.Awaitable`.

`typing.Coroutine`.

The base class `typing.Sequence` from the following classes with the same attribute set:

```
typing.ByteString
typing.Sequence
```

`typing.MutableSequence`

`typing.Mapping`.

`typing.MutableMapping`.

`typing.AbstractSet`, which also subsumes:

```
typing.KeysView
typing.ItemsView
```

`typing.MutableSet`.

`typing.Callable`, which also subsumes:

```
types.WrapperDescriptorType
types.MethodDescriptorType
types.MethodWrapperType

builtins.staticmethod

builtins.classmethod 
```

Skipped:

```
typing.TypedDict
typing._TypingEllipsis
typing._TypingEmpty
typing.NamedTuple

typing.NewType

typing._Final

numbers.Number
typing.Hashable

typing.Annotated

builtins.super

builtins.property

typing._ProtocolMeta
```

------

`builtins.memoryview`.

------

`builtins.object`.

------

`builtins.complex`.

`builtins.float`.

The base class `builtins.int` from the following classes with the same attribute set:

```
builtins.bool
builtins.int
```

`numbers.Complex`.

`numbers.Real`.

`numbers.Rational`.

`numbers.Integral`.

------

Skipped:

```
typing._BaseGenericAlias

typing._AnnotatedAlias

typing._TupleType
typing._SpecialGenericAlias

typing._UnionGenericAlias
typing._CallableGenericAlias
typing._GenericAlias
typing._LiteralGenericAlias
typing._ConcatenateGenericAlias

typing._LiteralSpecialForm 

typing._CallableType

collections.abc._CallableGenericAlias
```

------

Skipped:

```
typing.typing.io

typing.typing.re
```

------

`typing.SupportsIndex`.

`typing.SupportsBytes`.

`typing.SupportsComplex`.

`typing.SupportsFloat`.

`typing.SupportsInt`.

`typing.SupportsRound`.

`typing.SupportsAbs`.

------

`typing.TextIO`.

The base class `typing.IO` from the following classes with the same attribute set:

```
typing.BinaryIO
typing.IO
```

------

Skipped:

```
typing.ParamSpecKwargs
typing.ParamSpecArgs 

typing.TypeVar 

typing._TypeVarLike 

typing._Immutable 

typing.Protocol 

typing.ParamSpec 

typing.Generic 

typing._SpecialForm 

typing.ForwardRef
```


In [28]:
for attribute_frozenset, typeshed_class_set in attribute_frozenset_to_typeshed_class_set.items():
    print(typeshed_class_set_to_string(typeshed_class_set), '\n')

builtins.OverflowError
builtins.BufferError
builtins.KeyboardInterrupt
builtins.Exception
builtins.RecursionError
builtins.DeprecationWarning
builtins.ImportWarning
builtins.ValueError
builtins.ReferenceError
builtins.KeyError
builtins.ArithmeticError
builtins.Warning
builtins.PendingDeprecationWarning
builtins.EncodingWarning
builtins.BytesWarning
builtins.LookupError
builtins.NotImplementedError
builtins.UnicodeError
builtins.RuntimeError
builtins.IndexError
builtins.UserWarning
builtins.AssertionError
builtins.MemoryError
builtins.ResourceWarning
builtins.BaseException
builtins.SystemError
builtins.FloatingPointError
builtins.UnicodeWarning
builtins.SyntaxWarning
builtins.StopAsyncIteration
builtins.TypeError
builtins.GeneratorExit
builtins.FutureWarning
builtins.EOFError
builtins.RuntimeWarning
builtins.ZeroDivisionError 

builtins.set 

builtins.enumerate
builtins.filter
typing.Iterator
builtins.zip
builtins.map 

builtins.OSError
builtins.NotADirectoryError
builtins.PermissionErr